In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch

In [27]:
import mthree

In [4]:
service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")

## Create circuits

In [7]:
qubits = [4, 5, 6]
circuits = []

for num_qubits in qubits:
    qc = QuantumCircuit(num_qubits)
    qc.h(0)
    for q in range(1, num_qubits):
        qc.cx(0, q)
    qc.measure_all()
    
    circuits.append(qc)

## Transpilation

In [8]:
trans_qcs = transpile(circuits, backend=backend, optimization_level=3)

## Run

In [9]:
m3_mitigators = []
jobs = []
with Batch(backend=backend) as batch:
    sampler = Sampler(session=batch)
    
    for circuit in trans_qcs:    
        job = sampler.run([circuit])
        jobs.append(job)
        
        # same as before. just supply `sampler` instead of `backend`
        m3_mit = mthree.M3Mitigation(system=sampler)
        
        # run the measurement calibration
        # `async_cal=True` makes the cals non-blocking so that
        # next set of <main job> and <cals job> will be submitted immediately
        # without waiting for the previous set to finish
        final_layout = circuit.layout.final_index_layout(filter_ancillas=True)
        m3_mit.cals_from_system(qubits=final_layout, async_cal=True)
        
        # save the M3mitigation object to apply correction after all HW runs are done
        # applying correction later for non-iterative jobs may save time
        m3_mitigators.append(m3_mit)

## Post-process

In [26]:
for idx, job in enumerate(jobs):
    raw_counts = job.result()[0].data.meas.get_counts()
    print(f">> Job {idx}")
    print(f"raw counts {raw_counts}\n")

    quasi_prob = m3_mitigators[idx].apply_correction(
        counts=raw_counts,
        qubits=trans_qcs[idx].layout.final_index_layout(filter_ancillas=True),
        distance=6, # to save time for circuits with many qubits
    )
    print(f"mitigated quasi prob {quasi_prob.nearest_probability_distribution()}\n\n")

>> Job 0
raw counts {'1111': 1729, '0111': 51, '0000': 1847, '1101': 159, '1011': 77, '0010': 109, '0100': 55, '0110': 7, '1001': 14, '0101': 10, '1110': 17, '1100': 1, '1010': 2, '0011': 3, '1000': 15}

mitigated quasi prob {'0110': 0.0009172780678962695, '0101': 0.0011848151634769452, '1001': 0.0021350313338795507, '0111': 0.002976714064684803, '0100': 0.007059469332903917, '1011': 0.009252621521753135, '0010': 0.0232327360522948, '1101': 0.033419728140637976, '1111': 0.45434085374662897, '0000': 0.4654806988405825}


>> Job 1
raw counts {'00010': 348, '10000': 23, '11111': 1344, '00000': 1376, '11101': 417, '01111': 81, '11011': 88, '00100': 110, '01001': 9, '11110': 8, '11001': 94, '00110': 81, '10111': 17, '01110': 8, '10001': 8, '01101': 19, '01000': 11, '10011': 2, '10100': 9, '01011': 11, '00011': 1, '00001': 2, '10110': 7, '10010': 9, '01100': 1, '00111': 2, '10101': 3, '11000': 2, '01010': 2, '11100': 3}

mitigated quasi prob {'10010': 0.0007423601584763601, '10000': 0.000787